In [1]:
import pandas as pd
import numpy as np

from architecture import *
from typing import Optional

In [2]:
# df = pd.read_excel("data/occup.xlsx")
# df = df[3:]
# df = df.rename(columns=df.iloc[0]).drop(df.index[0])
# df.to_csv("data/occup.csv", index=False)
# df

In [3]:
df = pd.read_csv("data/occup.csv")
df

,année académique plan,semestre plan,code matière,matière,section responsable,type d'enseignement,nb étudiants,jour,heure début séance,heure fin séance,espace (code),nb.places dans l'espace,nombre total d'espaces,nombre total de places,overbooking
0,2023-2024,HIVER,AR-101,Théorie et critique du projet BA1,Architecture,Cours,251,monday,10:15,12:00,CE4,233.0,1,233.0,oui
1,2023-2024,HIVER,AR-114,Modélisation numérique,Architecture,Cours,257,tuesday,15:15,18:00,CO3,218.0,1,218.0,oui
2,2023-2024,HIVER,AR-119,Art fundaments,Architecture,Cours,249,wednesday,08:15,09:00,AAC008,60.0,6,539.0,non
3,2023-2024,HIVER,AR-119,Art fundaments,Architecture,Cours,249,wednesday,08:15,09:00,AAC014,60.0,6,539.0,non
4,2023-2024,HIVER,AR-119,Art fundaments,Architecture,Cours,249,wednesday,08:15,09:00,AAC108,60.0,6,539.0,non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,2023-2024,HIVER,UNIL-160,Chimie générale avancée (pour SFU),Chimie et génie chimique,Exercices,244,tuesday,10:15,12:00,CE1100,64.0,4,334.0,non
2151,2023-2024,HIVER,UNIL-160,Chimie générale avancée (pour SFU),Chimie et génie chimique,Exercices,244,tuesday,10:15,12:00,CM1120,100.0,4,334.0,non
2152,2023-2024,HIVER,UNIL-160,Chimie générale avancée (pour SFU),Chimie et génie chimique,Exercices,244,tuesday,10:15,12:00,GCD0386,42.0,4,334.0,non
2153,2023-2024,HIVER,UNIL-160,Chimie générale avancée (pour SFU),Chimie et génie chimique,Exercices,244,tuesday,10:15,12:00,INF1,128.0,4,334.0,non


In [6]:
def unique(list1):
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list


def entry_to_lecture(entry) -> Optional[Session]:
    num_participants = entry["nb étudiants"]
    if not np.isnan(num_participants) and not np.isnan(entry["nombre total de places"]):
        session = Session(
                type=SessionType.Lecture if entry["type d'enseignement"] == "Cours" else SessionType.Other,
                code=entry["code matière"],
                title=entry["matière"],
                num_participants=int(num_participants),
                allocated_room=Room(int(entry["nb.places dans l'espace"] if not np.isnan(entry["nb.places dans l'espace"]) else 0), entry["espace (code)"]),
                timeslot=str_to_timeslot(entry["jour"], entry["heure début séance"], entry["heure fin séance"]),
                already_swapped=False,
                overbooked=entry["overbooking"] == "oui"
        )
        return session
    return None


def str_to_timeslot(weekday_str, start_time, end_time) -> Timeslot:
    return Timeslot(weekday_str, (int(start_time[:2]), int(end_time[:2])))

In [7]:
all_sessions = []
for i in range(len(df)):
    entry = df.iloc[i]
    maybe_lect = entry_to_lecture(entry)
    if maybe_lect:
        all_sessions.append(maybe_lect)

all_sessions

[Session(type=<CourseType.Lecture: 1>, code='AR-101', title='Théorie et critique du projet BA1', num_participants=251, allocated_room=Room(capacity=233, name='CE4'), timeslot=Timeslot(day='monday', time=(10, 12)), already_swapped=False, overbooked=True),
 Session(type=<CourseType.Lecture: 1>, code='AR-114', title='Modélisation numérique', num_participants=257, allocated_room=Room(capacity=218, name='CO3'), timeslot=Timeslot(day='tuesday', time=(15, 18)), already_swapped=False, overbooked=True),
 Session(type=<CourseType.Lecture: 1>, code='AR-119', title='Art fundaments', num_participants=249, allocated_room=Room(capacity=60, name='AAC008'), timeslot=Timeslot(day='wednesday', time=(8, 9)), already_swapped=False, overbooked=False),
 Session(type=<CourseType.Lecture: 1>, code='AR-119', title='Art fundaments', num_participants=249, allocated_room=Room(capacity=60, name='AAC014'), timeslot=Timeslot(day='wednesday', time=(8, 9)), already_swapped=False, overbooked=False),
 Session(type=<Cours